In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import os
import numpy as np

In [5]:
os.getcwd()

'/Users/bipinkarki/Desktop/ML-ABKS-Music-Recommender-System/Notebooks'

In [6]:
tracks = pd.read_csv(r"datasets/data.csv")

In [7]:
tracks.tail(30)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
170623,0.5580,2020,0.175000,['Brent Faiyaz'],0.743,215058,0.54800,0,0dJhXJKZTpiaxTUc7uItIN,0.000180,6,0.0880,-6.545,0,Let Me Know,65,2020-02-07,0.0947,85.017
170624,0.6970,2020,0.052900,['Ozuna'],0.735,217240,0.79100,0,5Jm4w8jmPEBTLjI9vH4fXo,0.000146,6,0.1450,-4.462,1,Caramelo,80,2020-09-04,0.0679,168.087
170625,0.7700,2020,0.035800,"['Quinn XCII', 'Logic']",0.822,212013,0.83400,1,73681b7Hc0THHGZrSzl75y,0.000000,6,0.0461,-2.879,0,A Letter To My Younger Self (feat. Logic),65,2020-07-10,0.0663,115.006
170626,0.1550,2020,0.062700,['Cookiee Kawaii'],0.778,84000,0.57600,1,4ZAQiu61otvHVveuTsPAUr,0.004400,10,0.1160,-8.698,1,Vibe (If I Back It Up),68,2020-08-28,0.3010,80.027
170627,0.6290,2020,0.410000,"['Rauw Alejandro', 'Myke Towers', 'Sky Rompien...",0.809,184817,0.62400,1,10u2iOS0r6plfb9GknD7g4,0.000000,8,0.1080,-4.963,0,Ponte Pa' Mi,74,2020-04-16,0.1550,85.011
170628,0.3230,2020,0.000206,['Miley Cyrus'],0.407,290768,0.87200,0,6IsiCdn42x5fGWTUqkyDwj,0.045100,7,0.1270,-5.001,1,Zombie (Live from the NIVA Save Our Stages Fes...,71,2020-10-23,0.0354,164.000
170629,0.6910,2020,0.040600,"['Rauw Alejandro', 'Camilo']",0.721,222680,0.63700,0,4G3PTss3mU33Tau7t4KbwE,0.000000,9,0.0525,-5.134,1,Tattoo - Remix with Camilo,71,2020-11-13,0.2860,193.922
170630,0.6590,2020,0.002480,"['DVBBS', 'Quinn XCII']",0.714,150819,0.79800,0,4fCE1SCDulouoac9Q8XroU,0.000678,5,0.1030,-4.239,0,West Coast (feat. Quinn XCII),67,2020-10-16,0.0612,119.017
170631,0.3270,2020,0.217000,['YoungBoy Never Broke Again'],0.485,156077,0.81400,1,3qHgGyJY4GpXNOK4WL4NSo,0.000000,9,0.1120,-3.907,1,Red Eye,63,2020-02-21,0.1380,159.894
170632,0.7380,2020,0.160000,"['Rauw Alejandro', 'Dalex', 'Lenny Tavárez', '...",0.788,325692,0.62900,1,3oCghu20Kto5Xcu8Pg0cit,0.000828,0,0.0598,-4.727,1,"Elegí (feat. Farruko, Anuel AA, Sech, Dímelo F...",75,2020-08-28,0.0883,172.044


In [8]:
tfidf = TfidfVectorizer( stop_words='english')

#Replace NaN with an empty string
tracks['name'] = tracks['name'].fillna('')

tracks = tracks.head(25000)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(tracks['name'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(25000, 15701)

In [9]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:

indices = pd.Series(tracks.index, index=tracks['name']).drop_duplicates()


In [11]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    track_indices = [i[0] for i in sim_scores]
    print(tracks['name'].iloc[track_indices].tolist())
    return tracks['id'].iloc[track_indices].tolist()

In [12]:
get_recommendations("I Might Fall Back On You")

['Fall Back Down', 'Fall For You', 'I Might Fall Back On You', 'When I Fall In Love', 'When I Fall In Love', 'When I Fall In Love', 'When I Fall In Love', 'If I Ever Fall In Love', 'If I Ever Fall In Love', 'I Could Fall In Love']


['3pdHJCTk85ls2SGGXIJ7XH',
 '7lWF2mVr1KKbVnaT2nSlPo',
 '6MMF38qO03XJ7puIxVl8Gg',
 '6s6h2XK7Nl8lEcTzr7ezeB',
 '0w7mhiHnM0rhdu3l5mDDYl',
 '1Ame30MZ6yKnqaWFAch1m7',
 '1gMMY095TNREZ8ZZYsfM5L',
 '1jCsCYgzQQHk3bDJDuFbNi',
 '0uOPGU4CbYxzFxn6T7sblW',
 '3ACQkIMv6nGYHRpPGyS4BK']

In [13]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []


In [14]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [15]:
features = ['artist-name','popularity', 'genre', 'instruments']
#features = ['artists']
for feature in features:
    tracks[feature] = tracks[feature].apply(clean_data)
#     tracks[feature] = tracks[feature].apply(eval)

KeyError: 'artist-name'

In [ ]:
tracks.head(3)

In [ ]:
def cos_sim(a,b):
    dot_product = np.dot(a,b)
    norm_a = linalg.norm(a)
    norm_b = linalg.norm(b)
    return dot_product/(norm_a*norm_b)

In [ ]:
https://towardsdatascience.com/how-to-build-an-amazing-music-recommendation-system-4cce2719a572

In [ ]:
from collections import defaultdict
from scipy.spatial.distance import cdist
import difflib
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=20, 
                                   verbose=2))],verbose=True)
X = tracks.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
tracks['cluster_label'] = song_cluster_labels

In [ ]:
tracks.to_csv('tracks_with_cluster.csv')

In [ ]:
tracks.head()

In [ ]:
from sklearn.decomposition import PCA

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = tracks['name']
projection['cluster'] = tracks['cluster_label']

In [ ]:
import plotly.express as px
fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

In [ ]:
number_cols = ['time_signature','duration_ms','popularity','valence', 'acousticness', 'danceability',  'energy', 'instrumentalness', 'explicit','key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo']

In [ ]:
def get_song_data(song, spotify_data):
    
    """
    Gets the song data for a specific song. The song argument takes the form of a dictionary with 
    key-value pairs for the name and release year of the song.
    """
    
    try:
        song_data = spotify_data[(spotify_data['name'] == song['name']) ].iloc[0]
        return song_data
    
    except IndexError:
        return find_song(song['name'])

In [ ]:
def get_mean_vector(song_list, spotify_data):
  
    """
    Gets the mean vector for a list of songs.
    """
    
    song_vectors = []
    
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)

In [ ]:
def flatten_dict_list(dict_list):
   
    """
    Utility function for flattening a list of dictionaries.
    """
    
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict

In [ ]:
def recommend_songs(song_id, n_songs=10):
  
    """
    Recommends songs based on a list of previous songs that a user has listened to.
    """
    spotify_data = tracks
    song_name = tracks.loc[tracks.get("track-id") == song_id, 'name']
    print(song_name)
    song_list = [{'name': song_name}]
    print(song_list)
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, spotify_data)
    scaler = song_cluster_pipeline.steps[0][1]
    print(scaler)
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs['id'].tolist()

In [ ]:
recommend_songs('9975')

In [ ]:
df = pd.read_csv(r"C:/Users/shris/Desktop/profiles.csv")
print(df.gender)
df.columns = ['userid', 'gender', 'age', 'country']

In [ ]:
df.head()

In [ ]:
songs = df.gender.tolist()
output = {}
outputs = []
for song in songs:
    k = recommend_songs(song)
    output.update( {song : k})


In [ ]:
output

In [ ]:
df1 = pd.DataFrame.from_dict(output, orient='index')

In [ ]:
df1.to_csv('content_based.csv')